# Generate Test Dataset

증권사 리포트를 파싱한 데이터를 기반으로 RAG의 성능을 측정하기 위한 테스트 데이터셋 생성을 목표

Openai GPT의 도움을 받아 생성하는 방법 사용

In [1]:
import os, json
from dotenv import load_dotenv

import joblib

import pandas as pd
from openai import OpenAI # openai==1.52.2
import traceback

In [2]:
load_dotenv() ## OPEN_API_KEY 속성이 존재해야함

True

In [3]:
# 2. chunk된 텍스트 데이터를 담은 배열 예시
chunked_texts = [
    "이 텍스트는 인공지능의 역사와 발전 과정에 대해 설명합니다. 초기 AI 연구부터 최신 딥러닝 기술까지 다양한 사례를 다룹니다.",
    "이 문서는 데이터 전처리의 중요성과 여러 전처리 기법들을 설명하며, 실제 예시와 코드 스니펫을 포함하고 있습니다."
]


In [4]:
client = OpenAI()


In [19]:
def generate_qa_pairs(text_chunk):
    """
    주어진 json list 형식의 청크 리스트트에 대해 질답 쌍과 관련성 점수를 생성하는 프롬프트를 구성하여
    ChatGPT API를 호출한 후 결과를 반환합니다.
    
    출력은 JSON 형식의 문자열로, 각 질답 쌍에는 question, answer, document_id 항목이 포함됩니다.
    """

    prompt = f"""
    금융 도메인에 대한 QA 쌍을 생성하는 것이 너의 목표야.

    아래에 주어진 청크 리스트를 읽고, 각 청크의 "page_content" 내용을 기반으로 QA 쌍을 생성해줘.
    QA 쌍은 아래 세 가지 질문 유형 각각에 대해 1문제씩 만들어야 해.
    질문은 해당 청크의 정보를 기반으로 작성하고, 답변은 아래 출력 형식을 준수하는 JSON 객체로 만들어줘.
    또한, 생성된 QA 쌍마다 생성했을 때 참조한 문서의 ID 리스트를 "answer_document_id" 항목에 포함시켜줘.

    질문 유형:
    1. 상식적 질문 (General Knowledge Questions)
    - 예: "애플(Apple)의 창립 연도는?"
    2. 수치 기반 질문 (Numeric Questions)
    - 예: "2022년 애플의 총수익은 얼마인가?"
    3. 추론 질문 (Inference Questions)
    - 예: "애플의 순이익이 작년 대비 증가했는가?"

    출력 형식:
    {{
    "qa_pairs": [
        {{
        "question": "질문 내용",
        "answer": "답변 내용",
        "answer_document_id": [123]  // 정답 컨텍스트의 id 리스트 (id 리스트 형식으로 작성)
        }},
        ...
    ]
    }}

    청크 리스트:
    {text_chunk}
        """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 실제 사용 모델 이름으로 변경
            messages=[
                {"role": "system", "content": "너는 금융 도메인의 전문가야."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_completion_tokens=500,
            timeout=60
        )
        
        # API 응답에서 메시지 내용 추출
        print("----Response:", response)
        content = response.choices[0].message.content
        print("API 응답:", content)
        # JSON 파싱 시도 (모델이 형식을 잘 지켰다면 파싱 가능)
        return content
    except Exception as e:
        print("API 호출 중 오류 발생:", e)
        traceback.print_exc()
        return None


In [13]:
total_chunks = joblib.load('../data/final_documents.pkl')

In [14]:
print("청크 개수:", len(total_chunks))
print("첫 번째 청크 예시:", total_chunks[0])


청크 개수: 3629
첫 번째 청크 예시: page_content='20240509에 미래에셋증권에서 발행한 크래프톤에 관한 레포트에서 나온 내용.

Brief Description: Equity Research
2024.5.9

| 투자의견(유지) | 매수 |
| --- | --- |
| 목표주가(상향) ▲ | 370,000원 |
| 현재주가(24/5/8) | 260,000원 |
| 상승여력 | 42.3% |
| 영 업이익(24F,십억원) | 801 |
| Consensus 영업이익(24F,십억원) | 777 |
| EPS 성장률(24F,%) | 25.1 |
| MKT EPS 성장률(24F,%) | 75.9 |
| P/E(24F,x) | 17.0 |
| MKT P/E(24F,x) | 11.1 |
| KOSPI | 2,745.05 |
| 시가총액(십억원) | 12,575 |
| 발행주식수(백만주) | 48 |
| 유동주식비율(%) | 58.8 |
| 외국인 보유비중(%) | 36.5 |
| 베타(12M) 일간수익률 | 0.66 |
| 52주 최저가(원) | 146,500 |
| 52주 최고가(원) | 260,000 |
| (%) 1M 절대주가 6.8 | 6M 12M 36.3 28.1 |
| 상대주가 |  |
| 5.7 | 20.2 17.3 |
| 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 | 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 |
' metadata={'category': 'table', 'coordinates': [{'x': 0.041, 'y': 0.2669}, {'x': 0.2923, 'y': 0.2669}, {'x': 0.2923, 'y': 0.7877}, {'x': 0.041, 'y': 0.7877}], 'page': 1, 'id': 1, 'company_name': '크래프톤', 'report_date'

In [15]:
partial_chunks = {}
for chunk in total_chunks:
  text = chunk.page_content
  metadata = chunk.metadata
  
  company_name = metadata['company_name']
  if company_name in partial_chunks:
    partial_chunks[company_name].append(chunk)
  else :
    partial_chunks[company_name] = []
    

    

In [23]:
error_num = 0
json_error_num = 0
idx = 0

qa_pairs = []
for company_name, chunks in partial_chunks.items():
  print(f"\n--- {company_name} ---")
  # if idx == 1 : break
  
  file_dict = {}
  for chunk in chunks:
    dict_chunk = {}
    # dict_chunk['metadata'] = chunk.metadata
    dict_chunk['page_content'] = chunk.page_content
    dict_chunk['answer_document_id'] = chunk.metadata['nid']
    dict_chunk['source_file'] = chunk.metadata['source_file']
    dict_chunk['metadata'] = chunk.metadata
    if file_dict.get(chunk.metadata['source_file']) is None :
        file_dict[chunk.metadata['source_file']] = [dict_chunk]
    else :
        file_dict[chunk.metadata['source_file']].append(dict_chunk)
    
  for source_file, chunks in file_dict.items():
    chunk_list = []
    for chunk in chunks:
      dict_chunk = {}
      # dict_chunk['metadata'] = chunk['metadata']
      dict_chunk['page_content'] = chunk['page_content']
      dict_chunk['answer_document_id'] = chunk['metadata']['nid']
      chunk_list.append(dict_chunk)
    
    print("chunk list len", len(chunk_list))
    # for i in chunk_list:
    #     print("len of chunk",len(json.dumps(i)))
    # print("len of chunk",len(json.dumps(chunk_list[0])))
    json_chunk = json.dumps(chunk_list, ensure_ascii=False)
    # print(json_chunk)
      
    qa_result = generate_qa_pairs(json_chunk)
    cleaned_text = qa_result.removeprefix("```json").removesuffix("```").strip()
    try :
      qa_result = json.loads(cleaned_text)
      for qa in qa_result['qa_pairs']:
        print("qa['answer_document_id']", qa['answer_document_id'])
        qa['document_context'] = total_chunks[qa['answer_document_id'][0]-1].page_content
        qa['company_name'] = company_name
        qa['source_file'] = source_file
        
      print("QA 결과:", qa_result)
      if qa_result is not None:
          print(json.dumps(qa_result, indent=2, ensure_ascii=False))
          qa_pairs.extend(qa_result['qa_pairs'])
      else:
          print("결과 생성 실패")
          error_num += 1
    except Exception as e:
      print("----------------json parsing error---------------------------")
      print(cleaned_text)
      print(e)
      traceback.print_exc()
      json_error_num += 1
    
    
        
  idx += 1



--- 크래프톤 ---
chunk list len 27
----Response: ChatCompletion(id='chatcmpl-Ax0IH7du5TBRKxZwWzYEDYkBIQ8On', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n    "qa_pairs": [\n        {\n            "question": "크래프톤의 2024년 목표주가는 얼마인가?",\n            "answer": "370,000원입니다.",\n            "answer_document_id": [6]\n        },\n        {\n            "question": "크래프톤의 1분기 매출액은 얼마였으며, 이는 전년 대비 몇 퍼센트 증가했는가?",\n            "answer": "1분기 매출액은 6,660억원이며, 이는 전년 대비 24% 증가했습니다.",\n            "answer_document_id": [3]\n        },\n        {\n            "question": "크래프톤의 영업이익이 2023년과 2024년에는 증가할 것으로 예상되는데, 그 이유는 무엇인가?",\n            "answer": "2024년에는 다크앤다커M의 정식 출시와 함께 높은 트래픽이 유지되며 PUBG 매출 호조가 예상되기 때문입니다.",\n            "answer_document_id": [7]\n        }\n    ]\n}\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738625845, model='gpt-4o-mini-2024-07-18', object='chat.completion'

In [24]:
print("생성된 QA 쌍 개수:", len(qa_pairs))
print("오류 발생 횟수:", error_num)
print("JSON 파싱 오류 횟수:", json_error_num)
print("QA 쌍 예시:", qa_pairs[0])
qa_pairs_df = pd.DataFrame(qa_pairs)
qa_pairs_df.to_csv('../data/report_qa_pairs.csv', index=False)

생성된 QA 쌍 개수: 297
오류 발생 횟수: 0
JSON 파싱 오류 횟수: 0
QA 쌍 예시: {'question': '크래프톤의 2024년 목표주가는 얼마인가?', 'answer': '370,000원입니다.', 'answer_document_id': [6], 'document_context': '20240509에 미래에셋증권에서 발행한 크래프톤에 관한 레포트에서 나온 내용.\n\n목표주가 370,000원으로 상향. 업종 내 Top Pick 유지', 'company_name': '크래프톤', 'source_file': '크래프톤_20240509_미래에셋증권.pdf'}
